In [1]:
from bs4 import BeautifulSoup
import requests
import re
from selenium import webdriver

In [2]:
browser = webdriver.Firefox(executable_path="webdriver/geckodriver.exe")

c:\users\work\anaconda3\envs\kia\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [2]:
browser = webdriver.Chrome("webdriver/chromedriver.exe")

c:\users\work\anaconda3\envs\kia\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [6]:
main_url = 'https://edition.cnn.com'
# r = requests.get(url)
browser.implicitly_wait(5)
browser.get(main_url)
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [7]:
main_html = browser.page_source
main_soup = BeautifulSoup(main_html)

In [8]:
print("[INFO] Scraping categories and sub-categories ...")
list_cat_tag = main_soup.find_all("a",class_="sc-fjdhpX sc-chPdSV hnOkcW")
dict_cat_profile = {}
for i, cat in enumerate(list_cat_tag):
    if cat.text in dict_cat_profile.keys():
        continue
    cat_profile = {}
    cat_profile["url"] = cat["href"]
    
    subcat_tags = cat.find_next_sibling('ul')
    subcats = subcat_tags.find_all("a")
    list_subcat_href = {}
    for subcat in subcats:
        list_subcat_href[subcat.text]=subcat["href"]
        
    cat_profile["subcat"] = list_subcat_href
    
    dict_cat_profile[cat.text] = cat_profile

print(f"[INFO] found {len(dict_cat_profile)} categories: {[k for k in dict_cat_profile.keys()]}")

[INFO] Scraping categories and sub-categories ...
[INFO] found 13 categories: ['World', 'US Politics', 'Business', 'Health', 'Entertainment', 'Tech', 'Style', 'Travel', 'Sports', 'Videos', 'Features', 'Weather', 'More']


In [9]:
dict_cat_profile

{'World': {'url': '/world',
  'subcat': {'Africa': '/africa',
   'Americas': '/americas',
   'Asia': '/asia',
   'Australia': '/australia',
   'China': '/china',
   'Europe': '/europe',
   'India': '/india',
   'Middle East': '/middle-east',
   'United Kingdom': '/uk'}},
 'US Politics': {'url': '/politics',
  'subcat': {'The Biden Presidency': '/specials/politics/joe-biden-news',
   'Facts First': '/specials/politics/fact-check-politics',
   'US Elections': '/election/2022'}},
 'Business': {'url': '/business',
  'subcat': {'Markets': 'https://money.cnn.com/data/markets/',
   'Tech': '/business/tech',
   'Media': '/business/media',
   'Success': '/business/success',
   'Perspectives': '/business/perspectives',
   'Videos': '/business/videos'}},
 'Health': {'url': '/health',
  'subcat': {'Life, But Better': '/specials/health/life-but-better',
   'Fitness': '/specials/health/fitness-life-but-better',
   'Food': '/specials/health/food-life-but-better',
   'Sleep': '/specials/health/sleep-l

In [10]:
list_article_url = []
print("[INFO] Scraping article urls ...")
for k, v in dict_cat_profile.items():
    cat_name = k
    url = v["url"]
    # r = requests.get(url)
    browser.implicitly_wait(5)
    browser.get(main_url+url)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    html = browser.page_source
    soup = BeautifulSoup(html)
    article_div = soup.find_all(class_="cd__content")
    print(f"[INFO] Scraping from {main_url+url}: found {len(article_div)} articles")
    list_article_url.extend([a.find("a")["href"] for a in article_div])
    

[INFO] Scraping article urls ...
[INFO] Scraping from https://edition.cnn.com/world: found 112 articles
[INFO] Scraping from https://edition.cnn.com/politics: found 34 articles
[INFO] Scraping from https://edition.cnn.com/business: found 54 articles
[INFO] Scraping from https://edition.cnn.com/health: found 83 articles
[INFO] Scraping from https://edition.cnn.com/entertainment: found 40 articles
[INFO] Scraping from https://edition.cnn.com/business/tech: found 32 articles
[INFO] Scraping from https://edition.cnn.com/style: found 0 articles
[INFO] Scraping from https://edition.cnn.com/travel: found 0 articles
[INFO] Scraping from https://edition.cnn.com/sport: found 0 articles
[INFO] Scraping from https://edition.cnn.com/videos: found 0 articles
[INFO] Scraping from https://edition.cnn.com/specials: found 17 articles
[INFO] Scraping from https://edition.cnn.com/weather: found 31 articles
[INFO] Scraping from https://edition.cnn.com/more: found 35 articles


TypeError: 'NoneType' object is not subscriptable

In [17]:
def validate_article_url(url):
    valid = True
    if "http" in url:
        valid = False
        
    if "/video/" in url:
        valid = False
        
    if "/videos/" in url:
        valid = False
        
    if "/live-news/" in url:
        valid = False
        
    return valid

validate_article_url('/videos/politics/2022/07/12/jake-tapper-john-bolton-debate-january-6-coup-attempt-sot-lead-vpx.cnn/video/playlists/this-week-in-politics/')

False

In [18]:
list_valid_article_url = [url for url in list_article_url if validate_article_url(url)]

In [19]:
valid_article_url = list_valid_article_url[4]

In [20]:
print(main_url+valid_article_url)
r = requests.get(main_url+valid_article_url)
# browser.implicitly_wait(5)
# browser.get(main_url)
# browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
main_soup = BeautifulSoup(r.content, "html.parser")

https://edition.cnn.com/2022/07/14/europe/italy-government-collapse-july-intl/index.html


In [21]:
#title
main_soup.find("h1").text

"Italy's President Mattarella rejects Prime Minister Draghi's resignation "

In [22]:
#metadata
metadata_div = main_soup.find(class_="metadata")
print(metadata_div)
author_div = metadata_div.find(class_="metadata__byline__author")
author_name = author_div.text
author_url = author_div.find("a")
update_time = metadata_div.find(class_="update-time").text
author_name, author_url

<div class="metadata"><div class="metadata__info js-byline-images" data-bundle="byline"><p class="metadata__byline"><span class="metadata__byline__author">By Livia Borghese and Sharon Braithwaite, CNN</span></p><p class="metadata__show"></p><p class="update-time">Updated 2115 GMT (0515 HKT) July 14, 2022 <span class="video__source top_source" id="js-pagetop_video_source"></span></p></div></div>


('By Livia Borghese and Sharon Braithwaite, CNN', None)

In [23]:
list_related_article_urls = []
for p in main_soup.find_all(class_="zn-body__paragraph"):
    a_tags = p.find_all("a")
    for a in a_tags:
        list_related_article_urls.append(a['href'])
    print(p.text)

Rome (CNN)Italian Prime Minister Mario Draghi said Thursday that he would resign, after the 5-Star movement -- the largest party in the country's coalition government -- withdrew its support in a parliamentary confidence vote.
Draghi's measures -- a package designed to tackle Italy's cost-of-living crisis -- passed by 172 to 39. However, the 5-Star boycott leaves the government at real risk of collapse and could lead to an early election. 
After winning the vote but losing 5-Star's support, Draghi said in a statement: "I want to announce that this evening I will present my resignation to the President of the Republic." 
"Today's votes in Parliament are very significant from a political point of view. The majority of national unity that has supported this government since its creation is no longer there," he added. 
He had previously said that he would not lead a government that did not include 5-Star. 
However, Italy's President, Sergio Mattarella, has rejected Draghi's resignation, in

In [24]:
list_related_article_urls

['https://cnn.com/2019/06/06/business/ecb-president-mario-draghi/index.html',
 'https://cnn.com/2021/02/12/europe/mario-draghi-italy-prime-minister-intl/index.html']

In [3]:
import yaml

In [16]:
file_path = "src/backend/input/data.yaml"
with open(file_path, "r") as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [17]:
config

{'CNN': {'main_url': 'https://edition.cnn.com/',
  'main_category_tag': 'a',
  'main_category_class': 'sc-fjdhpX sc-chPdSV hnOkcW',
  'main_subcategory_tag': 'a',
  'main_subcatergory_class': 'sc-fjdhpX sc-kpOJdX cgygcP',
  'cat_post_tag': 'div',
  'cat_post_class': 'cd__content',
  'subcat_post_tag': 'div',
  'subcat_post_class': 'cd__content',
  'article_title_tag': 'h1',
  'article_title_class': 'pg-headline',
  'article_metadata_tag': 'div',
  'article_metadata_class': 'metadata',
  'article_metadata_author_tag': 'p',
  'article_metadata_author_class': 'metadata__byline__author',
  'article_metadata_datetime_tag': 'p',
  'article_metadata_datetime_class': 'update-time'}}

In [20]:
config["CNN"]

{'main_url': 'https://edition.cnn.com/',
 'main_category_tag': 'a',
 'main_category_class': 'sc-fjdhpX sc-chPdSV hnOkcW',
 'main_subcategory_tag': 'a',
 'main_subcatergory_class': 'sc-fjdhpX sc-kpOJdX cgygcP',
 'cat_post_tag': 'div',
 'cat_post_class': 'cd__content',
 'subcat_post_tag': 'div',
 'subcat_post_class': 'cd__content',
 'article_title_tag': 'h1',
 'article_title_class': 'pg-headline',
 'article_metadata_tag': 'div',
 'article_metadata_class': 'metadata',
 'article_metadata_author_tag': 'p',
 'article_metadata_author_class': 'metadata__byline__author',
 'article_metadata_datetime_tag': 'p',
 'article_metadata_datetime_class': 'update-time'}

In [87]:
s = "Updated 0615 GMT (1415 HKT) July 16, 2022"
t = re.compile("(\d{4})").search(s).span()
str_t = s[t[0]:t[1]]
str_t = str_t[:2] + ":" + str_t[2:]
str_t

'06:15'

In [88]:
d = re.compile("([a-z]{1,10} \d{1,2}, \d{4})").search(s.lower()).span()
str_d = s[d[0]:d[1]]
str_d

'July 16, 2022'

In [90]:
from datetime import datetime
datetime_object = datetime.strptime(f"{str_d} {str_t}", '%B %d, %Y %H:%M')

In [91]:
datetime_object

datetime.datetime(2022, 7, 16, 6, 15)